# Forward propagation

In [37]:
import numpy as np
import tensorflow as tf

In [123]:
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))
    
    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

X_train, Y_train = load_coffee_data()
norm = tf.keras.layers.Normalization(axis=-1)
norm.adapt(X_train)
X_train = norm(X_train)
m = X_train.shape[0] # number of training examples
n = X_train.shape[1] # number of features

In [156]:
class Dense:
    def __init__(self, units, activation=None):
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        self.W = None
        self.b = None
        self.built = False
    
    def build(self, input_shape):
        self.W = np.random.uniform(-1, 1,(input_shape, self.units))
        self.b = np.random.randint(-5,5,(self.units,))
        self.built = True
    
    def get_weights(self):
        return (self.W, self.b)
    
    def call(self, a_in):
        if self.built == False:
            raise Error("Layer has not been built yet")
        
        Z = np.dot(np.transpose(self.W), a_in) + self.b
        return self.activation(Z)

class Sequential:
    def __init__(self, layers):
        self.input_shape = None;
        self.layers = layers
        self.built = False;
        
    def build(self, X):
        a_in = X[0]
        size = self.input_shape or a_in.shape[0]
        for layer in self.layers:
            layer.build(size)
            a_in = layer.call(a_in)
            size = a_in.shape[0]
        self.built = True
     
    def forwardProp(self, x):
        a_in = x
        for layer in self.layers:
            a_in = layer.call(a_in)
        a_out = a_in    
        return a_out
    
    def predict(self, x):
        if not (self.built):
            raise Error('Model has not been built yet')
        return self.forwardProp(x)
    
    def __str__(self):
        print('\nSequential\n')
        for layer in self.layers:
            if layer.built:
                print('W shape:', layer.get_weights()[0].shape)
                print('b shape:', layer.get_weights()[1].shape)
                print('\n')
        
        return ''

model = Sequential([
    Dense(10, activation='relu'),
    Dense(5, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.build(X_train)
for i in range(10):
    print('t: {},  d: {}'.format(X_train[i][0], X_train[i][1]))
    prediction = model.predict(X_train[i])
    expected = Y_train[i]
    print('{} vs. {}'.format(0 if prediction < 0.5 else 1, int(expected[0])))
    print()

t: -0.8345550894737244,  d: -0.6528791785240173
1 vs. 1

t: 1.032305121421814,  d: -1.3851453065872192
1 vs. 0

t: 0.3089390993118286,  d: 0.8716251254081726
1 vs. 0

t: -1.083568811416626,  d: -1.515484094619751
1 vs. 0

t: -0.7894312739372253,  d: 0.6194934844970703
1 vs. 1

t: 0.18112416565418243,  d: -1.1790282726287842
1 vs. 1

t: -0.25681355595588684,  d: 0.6615495085716248
1 vs. 0

t: -0.29007601737976074,  d: 0.5335373282432556
1 vs. 0

t: 1.5498857498168945,  d: 0.7103531956672668
1 vs. 0

t: -0.39534124732017517,  d: -1.0471969842910767
1 vs. 1



In [140]:
def Dense(a_in, W, b):
    """
    Computes dense layer
    Args:
      a_in (ndarray (n, )) : Data, 1 example 
      W    (ndarray (n,j)) : Weight matrix, n features per unit, j units
      b    (ndarray (j, )) : bias vector, j units  
    Returns
      a_out (ndarray (j,))  : j units|
    """
    units = W.shape[1]
    a_out = np.zeros(units)
    for j in range(units):
        w = W[:,j]
        z = np.dot(w, a_in) + b[j]
        a_out[j] = g(z)
    
    return a_out

In [27]:
def Sequential(x, *args):
    a_prev = x;
    a_next = None;
    for W, b in args:
        a_next = Dense(a_prev, W, b)
        a_prev = a_next
    return a_next

In [28]:
W1_tmp = np.array( [[-8.93,  0.29, 12.9 ], [-0.1,  -7.32, 10.81]] )
b1_tmp = np.array( [-9.82, -9.28,  0.96] )
W2_tmp = np.array( [[-31.18], [-27.59], [-32.56]] )
b2_tmp = np.array( [15.41] )

In [29]:
for i in range(0, 10):
    predicted = None;
    y_hat = Sequential(X_train[i,:], (W1_tmp, b1_tmp), (W2_tmp, b2_tmp))
    if y_hat > 0.5:
        predicted = 1
    else:
        predicted = 0
    
    expected = Y_train[i]
    print('Predicted {}, expected {} -> {}'.format(predicted, expected, 
                                                   'correct' if predicted == expected 
                                                   else 'incorrect'))

Predicted 1, expected [1.] -> correct
Predicted 0, expected [0.] -> correct
Predicted 0, expected [0.] -> correct
Predicted 0, expected [0.] -> correct
Predicted 1, expected [1.] -> correct
Predicted 1, expected [1.] -> correct
Predicted 0, expected [0.] -> correct
Predicted 0, expected [0.] -> correct
Predicted 0, expected [0.] -> correct
Predicted 1, expected [1.] -> correct


In [30]:
w_1 = np.random.rand(X.shape[1], 1).reshape(X.shape[1],)
b_1 = np.random.randint(4)
layer_1 = [
    [w_1,b_1]
]
print(layer_1)
# a_1 = np.dot(layer_1[0][0], X[0]) + b_1 

w_2 = np.random.rand(w_1.shape[0], 1)
b_2 = np.random.randint(4)
layer_2 = [
    [w_2, b_2]
]
print(layer_2)

NameError: name 'X' is not defined

In [115]:
None or 5

5